In [1]:
#NDWI load packages

import datacube

import sys
sys.path.insert(1, '../Tools/')
from dea_tools.dask import create_local_dask_cluster
from dea_tools.plotting import rgb



In [2]:
# Import tools to mask data for cloud

import scipy.ndimage
import xarray
import numpy
import datacube
from datacube.utils.masking import make_mask
from datacube.utils.masking import mask_invalid_data
from odc.algo import mask_cleanup

import sys
sys.path.insert(1, '../Tools/')
from dea_tools.plotting import rgb


import rasterio
import xarray as xr

In [3]:
# connect to datacube

dc = datacube.Datacube(app="07_Parallel_processing_with_Dask")

In [4]:
#Enable Dask

client = create_local_dask_cluster(return_client=True)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/akjournalism@gmail.com/proxy/8787/status,
Dashboard: /user/akjournalism@gmail.com/proxy/8787/status,Workers: 1
Total threads: 2,Total memory: 12.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36877,Workers: 1
Dashboard: /user/akjournalism@gmail.com/proxy/8787/status,Total threads: 2
Started: Just now,Total memory: 12.21 GiB
Comm: tcp://127.0.0.1:37661,Total threads: 2
Dashboard: /user/akjournalism@gmail.com/proxy/45901/status,Memory: 12.21 GiB
Nanny: tcp://127.0.0.1:39371,


In [5]:
# Lazy load data LANDSAT

lazy_ds_Land = dc.load(product=["ga_ls5t_ard_3", "ga_ls8c_ard_3",
                  "ga_ls9c_ard_3"],
                  measurements=["nbart_green", "nbart_swir_2", "oa_fmask"],
                  x=(149.64, 149.94),
                  y=(-37.00, -37.2),
                  time=('2020-02-10', '2020-04-10'),
                  dask_chunks={"time": 1, "x": 2048, "y": 2048})
resampling={
                   "fmask": "nearest",
                   "*": "bilinear"
               },


lazy_ds_Land

<xarray.Dataset>
Dimensions:       (time: 12, y: 855, x: 986)
Coordinates:
  * time          (time) datetime64[ns] 2020-02-10T23:44:57.745417 ... 2020-0...
  * y             (y) float64 -4.153e+06 -4.153e+06 ... -4.178e+06 -4.178e+06
  * x             (x) float64 1.567e+06 1.567e+06 ... 1.597e+06 1.597e+06
    spatial_ref   int32 3577
Data variables:
    nbart_green   (time, y, x) int16 dask.array<chunksize=(1, 855, 986), meta=np.ndarray>
    nbart_swir_2  (time, y, x) int16 dask.array<chunksize=(1, 855, 986), meta=np.ndarray>
    oa_fmask      (time, y, x) uint8 dask.array<chunksize=(1, 855, 986), meta=np.ndarray>
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

In [6]:
# Lazy load data SENTINEL

lazy_ds_Sent = dc.load(product=["'ga_s2am_ard_3", "ga_s2bm_ard_3"],
                  measurements=["nbart_green", "nbart_swir_2", "oa_fmask"],
                  x=(149.46, 149.36),
                  y=(-37.2, -37.00),
                  time=('2020-01-14', '2020-04-14'),
                  resolution=(-30, 30),
                  dask_chunks={"time": 1, "x": 2048, "y": 2048})
resampling={
                   "fmask": "nearest",
                   "*": "bilinear"
               },
    
  
lazy_ds_Sent

<xarray.Dataset>
Dimensions:       (time: 40, y: 771, x: 395)
Coordinates:
  * time          (time) datetime64[ns] 2020-01-15T00:06:50.961837 ... 2020-0...
  * y             (y) float64 -4.149e+06 -4.149e+06 ... -4.172e+06 -4.172e+06
  * x             (x) float64 1.542e+06 1.542e+06 ... 1.554e+06 1.554e+06
    spatial_ref   int32 3577
Data variables:
    nbart_green   (time, y, x) int16 dask.array<chunksize=(1, 771, 395), meta=np.ndarray>
    nbart_swir_2  (time, y, x) int16 dask.array<chunksize=(1, 771, 395), meta=np.ndarray>
    oa_fmask      (time, y, x) uint8 dask.array<chunksize=(1, 771, 395), meta=np.ndarray>
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

In [7]:
# Merge XArrays

import xarray as xr

# Merge the XArrays
merged_xarr = xr.concat([lazy_ds_Land, lazy_ds_Sent], dim='merged_dim')

# Print the merged XArray
print(merged_xarr)

#merged_dim = 2 below could be a problem

<xarray.Dataset>
Dimensions:       (time: 52, y: 968, x: 1381, merged_dim: 2)
Coordinates:
  * time          (time) datetime64[ns] 2020-01-15T00:06:50.961837 ... 2020-0...
  * y             (y) float64 -4.178e+06 -4.178e+06 ... -4.149e+06 -4.149e+06
  * x             (x) float64 1.542e+06 1.542e+06 ... 1.597e+06 1.597e+06
    spatial_ref   int32 3577
Dimensions without coordinates: merged_dim
Data variables:
    nbart_green   (merged_dim, time, y, x) float64 dask.array<chunksize=(1, 1, 968, 1381), meta=np.ndarray>
    nbart_swir_2  (merged_dim, time, y, x) float64 dask.array<chunksize=(1, 1, 968, 1381), meta=np.ndarray>
    oa_fmask      (merged_dim, time, y, x) float64 dask.array<chunksize=(1, 1, 968, 1381), meta=np.ndarray>
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref


In [8]:
# app

dc = datacube.Datacube(app="Masking_data")

# flags

merged_xarr.oa_fmask.attrs["flags_definition"]

{'fmask': {'bits': [0, 1, 2, 3, 4, 5, 6, 7],
  'values': {'0': 'nodata',
   '1': 'valid',
   '2': 'cloud',
   '3': 'shadow',
   '4': 'snow',
   '5': 'water'},
  'description': 'Fmask'}}

In [9]:

merged_xarr.oa_fmask


<xarray.DataArray 'oa_fmask' (merged_dim: 2, time: 52, y: 968, x: 1381)>
dask.array<concatenate, shape=(2, 52, 968, 1381), dtype=float64, chunksize=(1, 2, 968, 1381), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 2020-01-15T00:06:50.961837 ... 2020-04...
  * y            (y) float64 -4.178e+06 -4.178e+06 ... -4.149e+06 -4.149e+06
  * x            (x) float64 1.542e+06 1.542e+06 ... 1.597e+06 1.597e+06
    spatial_ref  int32 3577
Dimensions without coordinates: merged_dim
Attributes:
    units:             1
    nodata:            0
    flags_definition:  {'fmask': {'bits': [0, 1, 2, 3, 4, 5, 6, 7], 'values':...
    crs:               EPSG:3577
    grid_mapping:      spatial_ref

In [10]:
dataset = xr.Dataset(merged_xarr)

In [11]:
merged_xarr


<xarray.Dataset>
Dimensions:       (time: 52, y: 968, x: 1381, merged_dim: 2)
Coordinates:
  * time          (time) datetime64[ns] 2020-01-15T00:06:50.961837 ... 2020-0...
  * y             (y) float64 -4.178e+06 -4.178e+06 ... -4.149e+06 -4.149e+06
  * x             (x) float64 1.542e+06 1.542e+06 ... 1.597e+06 1.597e+06
    spatial_ref   int32 3577
Dimensions without coordinates: merged_dim
Data variables:
    nbart_green   (merged_dim, time, y, x) float64 dask.array<chunksize=(1, 1, 968, 1381), meta=np.ndarray>
    nbart_swir_2  (merged_dim, time, y, x) float64 dask.array<chunksize=(1, 1, 968, 1381), meta=np.ndarray>
    oa_fmask      (merged_dim, time, y, x) float64 dask.array<chunksize=(1, 1, 968, 1381), meta=np.ndarray>
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

In [12]:
import numpy as np
import dask.array as da

def make_mask(variable, **flags):
    mask = da.zeros_like(variable, dtype=bool)
    mask_value = True

    for flag_name, flag_value in flags.items():
        mask |= variable == flag_value

    return mask == mask_value

In [13]:
# Create the mask based on "valid" pixels

clear_mask = make_mask(merged_xarr.oa_fmask, fmask="valid")



In [14]:
# Apply the mask

clear = merged_xarr.where(clear_mask)

In [ ]:
# Plot
clear_mask.plot(col="time", col_wrap=4)

In [15]:
# Identify pixels that are either "valid", "water" or "snow"

cloud_free_mask = (
    make_mask(merged_xarr.oa_fmask, fmask="valid") | 
    make_mask(merged_xarr.oa_fmask, fmask="water") | 
    make_mask(merged_xarr.oa_fmask, fmask="snow")
)

# Apply the mask
cloud_free = merged_xarr.where(cloud_free_mask)

In [16]:
# Set invalid nodata pixels to NaN

valid_data = merged_xarr.oa_fmask.plot
valid_data

# Identify pixels that are either "cloud" or "cloud_shadow"

cloud_shadow_mask = (
    make_mask(merged_xarr.oa_fmask, fmask="cloud") | 
    make_mask(merged_xarr.oa_fmask, fmask="shadow")
)

In [ ]:
# Plot cloud shadow mask

cloud_shadow_mask.plot(col="time", col_wrap=4)

In [17]:
# Dilate all cloud and cloud shadow pixels by 5 pixels in all directions

cloud_shadow_buffered = mask_cleanup(mask=cloud_shadow_mask,
                                     mask_filters=[("dilation", 5)])

In [ ]:
# Plot cloud shadow buffered

cloud_shadow_buffered.plot(col="time", col_wrap=4)

In [18]:
# Apply the mask

buffered_cloud_free = merged_xarr.where(~cloud_shadow_buffered)

# Set invalid nodata pixels to NaN

valid_data = mask_invalid_data(buffered_cloud_free)

valid_data

<xarray.Dataset>
Dimensions:       (time: 52, y: 968, x: 1381, merged_dim: 2)
Coordinates:
  * time          (time) datetime64[ns] 2020-01-15T00:06:50.961837 ... 2020-0...
  * y             (y) float64 -4.178e+06 -4.178e+06 ... -4.149e+06 -4.149e+06
  * x             (x) float64 1.542e+06 1.542e+06 ... 1.597e+06 1.597e+06
    spatial_ref   int32 3577
Dimensions without coordinates: merged_dim
Data variables:
    nbart_green   (merged_dim, time, y, x) float64 dask.array<chunksize=(1, 1, 968, 1381), meta=np.ndarray>
    nbart_swir_2  (merged_dim, time, y, x) float64 dask.array<chunksize=(1, 1, 968, 1381), meta=np.ndarray>
    oa_fmask      (merged_dim, time, y, x) float64 dask.array<chunksize=(1, 1, 968, 1381), meta=np.ndarray>
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

In [19]:
# calc mNDWI

band_diff = valid_data.nbart_green - valid_data.nbart_swir_2
band_sum = valid_data.nbart_green + valid_data.nbart_swir_2

valid_data["mndwi"] = band_diff / band_sum

valid_data

<xarray.Dataset>
Dimensions:       (time: 52, y: 968, x: 1381, merged_dim: 2)
Coordinates:
  * time          (time) datetime64[ns] 2020-01-15T00:06:50.961837 ... 2020-0...
  * y             (y) float64 -4.178e+06 -4.178e+06 ... -4.149e+06 -4.149e+06
  * x             (x) float64 1.542e+06 1.542e+06 ... 1.597e+06 1.597e+06
    spatial_ref   int32 3577
Dimensions without coordinates: merged_dim
Data variables:
    nbart_green   (merged_dim, time, y, x) float64 dask.array<chunksize=(1, 1, 968, 1381), meta=np.ndarray>
    nbart_swir_2  (merged_dim, time, y, x) float64 dask.array<chunksize=(1, 1, 968, 1381), meta=np.ndarray>
    oa_fmask      (merged_dim, time, y, x) float64 dask.array<chunksize=(1, 1, 968, 1381), meta=np.ndarray>
    mndwi         (merged_dim, time, y, x) float64 dask.array<chunksize=(1, 1, 968, 1381), meta=np.ndarray>
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

In [20]:
del buffered_cloud_free, cloud_shadow_buffered, cloud_free, clear_mask

In [21]:
# Call lazy data XArray from Dask 

mndwi_load = valid_data.mndwi.load()
mndwi_load

<xarray.DataArray 'mndwi' (merged_dim: 2, time: 52, y: 968, x: 1381)>
array([[[[        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         ...,
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan]],

        [[        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
...
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan]],

        [[        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         ...,
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan]]]])
Coordinates:
  * time         (time) datetime64[ns] 2020-01-15T00:06:50.961837 ... 2020-04...
  * y            (y) float64 -4.178e+06 -4.178e+06 ... -4.149e+06 -4.149e+06
  * x            (x) float64 1.542e+06 1.542e+06 ... 1.597e+06 1.597e+06
    spatial_ref  int32 3577
Dimensions without coordinates: merged_dim

In [ ]:
# plot data 

loaded_ds.mndwi.plot(col='time', col_wrap=5)

In [22]:
# create binary water not water data

import rasterio
import xarray as xr

# Extract the MNDWI data variable COULD BE WRONG WORKS WITHOUT
ndwi_variable = valid_data.mndwi

# Set the threshold
threshold = 0.3  # Adjust as per your requirement

# Create a binary water or no water raster
binary_data = ndwi_variable > threshold

#binary_data = binary_date.dtype('uint8')

In [23]:
# Calculate total wetness through time
wet_pixels_sum = binary_data.sum(dim=['time'])

In [24]:
# Calculate total count wet pixels **** CHECK WORKS ****
wet_pixels_count = binary_data.count(dim=['time'])

In [25]:
wet_pixels = wet_pixels_sum/wet_pixels_count

In [26]:
wet_pixels.attrs = valid_data.attrs

In [ ]:
#not necessary
wet_pixels.plot(size=5, cmap='Blues')

In [27]:
#wet_pixels = wo.isel(time=-0).to_array()

from datacube.utils.cog import write_cog

write_cog (geo_im=wet_pixels,
          fname='./Land_sent_bin_mNDWI_test3_.tif',
          overwrite=True)

PosixPath('Land_sent_bin_mNDWI_test3_.tif')

In [ ]:
#CLOSE DASK

client.close()

# delete stored objects

del valid_data, wet_pixels
